<a href="https://colab.research.google.com/github/imelike/Transformers_LLM/blob/main/Only_Model_Training_and_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import pandas as pd
import tensorflow as tf

In [2]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00
time: 271 µs (started: 2023-12-07 07:43:43 +00:00)


In [3]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
time: 12.4 s (started: 2023-12-07 07:43:43 +00:00)


In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
time: 2.22 s (started: 2023-12-07 07:43:55 +00:00)


In [5]:
!git lfs install

Git LFS initialized.
time: 205 ms (started: 2023-12-07 07:43:57 +00:00)


In [6]:
!git config --global credential.helper store

time: 106 ms (started: 2023-12-07 07:43:58 +00:00)


In [7]:
from huggingface_hub import notebook_login
notebook_login()

time: 381 ms (started: 2023-12-07 07:43:58 +00:00)


In [8]:
from transformers import AutoTokenizer

time: 5.3 s (started: 2023-12-07 07:43:58 +00:00)


In [9]:
# Tokenize edeceğim temizlediğim, hazır veri setini Hugging Face Hub'dan indireceğim

from datasets import load_dataset

reviews_sample = load_dataset("imelike/turkishReviews-ds")
reviews_sample

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/362520 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40281 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'review_length'],
        num_rows: 362520
    })
    validation: Dataset({
        features: ['review', 'review_length'],
        num_rows: 40281
    })
})

time: 7.32 s (started: 2023-12-07 07:44:15 +00:00)


In [10]:
# kendi ürettiğimiz Tokenizer()'ı Hugging Face Hub'dan indirelim

#downloaded_tokenizer = AutoTokenizer.from_pretrained("imelike/turkishReviews-ds-mini")
tokenizer = AutoTokenizer.from_pretrained("imelike/turkishReviews-ds")

tokenizer_config.json:   0%|          | 0.00/521 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/650k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.54M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

time: 1.42 s (started: 2023-12-07 07:44:22 +00:00)


In [11]:
# her bir yorum için alınacak token sayısı
context_length = 40

time: 666 µs (started: 2023-12-07 07:44:23 +00:00)


In [12]:
def tokenize(element):
    outputs = tokenizer(
        element["review"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = reviews_sample.map(
    tokenize, batched=True, remove_columns=reviews_sample["train"].column_names
)
tokenized_datasets

Map:   0%|          | 0/362520 [00:00<?, ? examples/s]

Map:   0%|          | 0/40281 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 359639
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 39992
    })
})

time: 1min 33s (started: 2023-12-07 07:44:29 +00:00)


In [13]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

time: 789 ms (started: 2023-12-07 07:46:17 +00:00)


In [14]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: (5, 40)
attention_mask shape: (5, 40)
labels shape: (5, 40)
time: 3.88 s (started: 2023-12-07 07:46:25 +00:00)


In [15]:
for key in out:
    print(f"{key}: {out[key][0]}")

input_ids: [   38   299  7292  1937 10290  3157  4003    14 41404  7292 10290    12
   345  1751   569  1616  1813  3980  1813  1361  1768    14  1505  1280
  1184   519  6773  1075 10185  1361   452  1204 10791    14  5906   963
    14  3085    14  7082]
attention_mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
labels: [   38   299  7292  1937 10290  3157  4003    14 41404  7292 10290    12
   345  1751   569  1616  1813  3980  1813  1361  1768    14  1505  1280
  1184   519  6773  1075 10185  1361   452  1204 10791    14  5906   963
    14  3085    14  7082]
time: 14.3 ms (started: 2023-12-07 07:46:29 +00:00)


In [16]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

time: 531 ms (started: 2023-12-07 07:46:29 +00:00)


In [17]:
len(tf_train_dataset)

11239

time: 7.54 ms (started: 2023-12-07 07:46:30 +00:00)


# **PART D:** Train a 🤗 Hugging Face Causal Language Model (Transformer) from scratch



In [18]:
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig

context_length = 40

time: 102 ms (started: 2023-12-07 07:46:36 +00:00)


In [19]:
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

time: 342 ms (started: 2023-12-07 07:46:37 +00:00)


In [20]:
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)  # Builds the model
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  125778432 
 er)                                                             
                                                                 
Total params: 125778432 (479.81 MB)
Trainable params: 125778432 (479.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
time: 4.86 s (started: 2023-12-07 07:46:40 +00:00)


In [21]:
from huggingface_hub import notebook_login

notebook_login()

time: 25.2 ms (started: 2023-12-07 07:46:46 +00:00)


In [22]:
# set up optimizer

from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

time: 6.23 ms (started: 2023-12-07 07:46:56 +00:00)


In [23]:
# compile the model

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

time: 22.8 ms (started: 2023-12-07 07:46:57 +00:00)


In [24]:
# Train the model


from transformers.keras_callbacks import PushToHubCallback

#callback = PushToHubCallback(output_dir="imelike/turkishReviews-ds-mini", tokenizer=tokenizer)
#callback = PushToHubCallback(output_dir="imelike/turkishReviews-ds", tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/imelike/turkishReviews-ds into local empty directory.


Download file tf_model.h5:   0%|          | 16.0k/480M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/480M [00:00<?, ?B/s]

time: 1min 49s (started: 2023-12-07 07:46:58 +00:00)


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=3, callbacks=[callback])

#model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=3)


Epoch 1/3
11239/11239 [==============================] - 5797s 514ms/step - loss: 6.2977 - val_loss: 5.4911
Epoch 2/3
 2835/11239 [======>.......................] - ETA: 1:08:18 - loss: 5.5204

In [ ]:
model.push_to_hub("imelike/turkishReviews-ds")